# Descritores dominantes e não dominantes

Ler um atributo por meio de uma instância devolve o atributo desta instância. O fallback é o atributo da classe.

Escrever/sobrescrever o valor de um atributo por meio de uma instância afeta apenas esta instância, sem fallback para a classe.

A assimetria está na implementação do método `__set__`.